In [ ]:
# Encode the CSV as utf-8 for python
import csv

path =  './training_set.csv'

with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open('training_set_encoded.csv', 'w') as outfile:
     inputs = csv.reader(infile)
     output = csv.writer(outfile)

     for index, row in enumerate(inputs):
#          # Create file with no header
#          if index == 0:
#              continue
#          print(row)
         output.writerow(row)

In [106]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [107]:
import nltk
# nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [108]:
# MacOS garbage
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [109]:
train_df = pd.read_csv('./training_set_encoded.csv')
train_df.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [110]:
train_sample_fraction = 0.3
y = train_df[['Engagements']]
# X = train_df.loc[:, train_df.columns != 'Engagements']
X = train_df[['Followers at Posting']]
# X.dtypes

In [115]:
rows = train_df.size
X['Year'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Month'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Day'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Hour'] = pd.DataFrame(0, index=range(rows), columns=range(1))

In [116]:
dates = train_df[['Created']]

for r in range(dates.values.size):
    row = dates.values[r].tolist()
    row = "".join(row).split()
    date = row[0].split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    hour = row[1].split(":")[0]
    X['Year'][r] = year
    X['Month'][r] = month
    X['Day'][r] = day
    X['Hour'][r] = hour

X.head()

,Followers at Posting,Year,Month,Day,Hour
0,36984682,2019,5,21,23
1,36984682,2019,5,21,22
2,36984682,2019,5,21,22
3,36984682,2019,5,21,22
4,36984682,2019,5,21,20


In [118]:
Media_Type = train_df[['Type']]

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Media_Type)
Media_Type = label_encoder.transform(Media_Type)
# print([x for x in Media_Type])
label_encoder.classes_

X['Type'] = Media_Type
X.tail()

,Followers at Posting,Year,Month,Day,Hour,Type
7761,24875542,2017,10,1,19,2
7762,24875542,2017,10,1,13,2
7763,24875542,2017,10,1,9,0
7764,24875542,2017,10,1,2,2
7765,24875542,2017,10,1,0,2


In [130]:
rows = train_df.size
X['neg'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['neu'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['pos'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['compound'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))

In [133]:
# Sentiment analysis on "Description" feature
description = train_df[['Description']]

analyser = vader.SentimentIntensityAnalyzer()

for r in range(description.values.size):
    row = description.values[r].tolist()
    if isinstance(row[0], str):
        snt = analyser.polarity_scores(row[0])
        X['neg'][r] = snt['neg']
        X['neu'][r] = snt['neu']
        X['pos'][r] = snt['pos']
        X['compound'][r] = snt['compound']

X.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.7/si

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound
0,36984682,2019,5,21,23,2,0.0,1.0,0.0,0.0
1,36984682,2019,5,21,22,2,0.0,1.0,0.0,0.0
2,36984682,2019,5,21,22,2,0.0,1.0,0.0,0.0
3,36984682,2019,5,21,22,2,0.0,1.0,0.0,0.0
4,36984682,2019,5,21,20,2,0.0,1.0,0.0,0.0


In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=train_sample_fraction, random_state=420)

xgb = XGBRegressor(
#     eta=0.3,
#     gamma=0.05,
#     max_depth=10,
#     min_child_weight=0.5,
#     max_delta_step=1,
#     reg_lambda=1,
#     alpha=1,
    objective='reg:linear',
    eval_metric='mae')

In [135]:
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [42]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [136]:
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print(MAPE)
MAPE_test = mean_absolute_percentage_error(y_test.head(), y_pred[:5])
# print(y_test.head())
# print(y_pred[:5])
print(MAPE_test)
MAE = mean_absolute_error(y_test, y_pred)
print(MAE)

37.96036246683822
49.55763363367163
57863.83662821889
